[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/scmopt/moai-manual/blob/main/optseq-trial.ipynb)


# AMPLPYパッケージの使用法

> AMPLをPythonから呼び出すためのAMPLPYパッケージの使用法

## ローカルで amplpyを動かす方法

AMPLをインストールしてampl.exeがある場所をEnvironmentで指定する。


In [ ]:
from amplpy import AMPL, Environment

env = Environment("/Users/mikiokubo/Documents/ampl/")
ampl = AMPL(env)

## Google Colab.で使う場合

ampl_notebook関数で、使うソルバー(modules)とlicense_uuidを設定する（空白でも動く）。

In [ ]:
!pip install amplpy
from amplpy import ampl_notebook
ampl = ampl_notebook(
    modules=["highs","gurobi","cbc","scip","coin"]
            #coin includes ipopt, couenne, bonmin
    license_uuid=None)

## AMPLPYライブラリ

AMPLをPythonから呼び出して使うためには amplpy（アンプルパイと読む）パッケージを使う。AMPLはモデルファイル(.mod)とデータファイル(.dat)とコマンドファイル(.run)から構成される。


## マジックコマンド writefile

マジックコマンド `%%writefile` の後にファイル名を入れてモデルファイルを出力しておく。

In [6]:
%%writefile production.mod
#%%writefile production.mod
var x >= 0; # 製品1の生産量
var y >= 0; # 製品2の生産量
maximize Profit: 40 * x + 30 * y;
subject to Constraint1: x + 2 * y <= 20; # 資源1の制約
subject to Constraint2: 3 * x + y <= 30; # 資源2の制約

Overwriting production.mod


## AMPLクラス

モデルインスタンス `ampl` に対しては、以下のメソッドが使える。

- `read`: モデルファイルを読み込む。
- `read_data`: データファイルを読み込む。
- `eval`: AMPLコマンドを実行する。
- `solve`: ソルバーを用いて求解する。
- `option`: オプションを設定する。
- `get_value`: 値を得る。
  
これらのメソッドを用いてソルバー指定、最適化や結果を得ることができる。

In [3]:
ampl.read("production.mod")
ampl.option["solver"] = "highs"
ampl.solve()
print("Profit=", ampl.get_value("Profit"))
print("x=", ampl.get_value("x"))
print("y=", ampl.get_value("y"))

HiGHS 1.10.0: optimal solution; objective 500
2 simplex iterations
0 barrier iterations
Profit= 500
x= 8
y= 6


## モデルとデータの分離

AMPLの1つの特徴としてモデルとデータの分離があげられる。以下は AMPLコマンドで普通に書いた例を示す。

セルの先頭でマジックコマンド`%%ampl_eval`と書いた後に、AMPLのコマンドを記述する。

In [7]:
%%ampl_eval
#%%ampl_eval
reset;
model;
param a;
param b;
param c;
var x >= 0;
var y >= 0;
maximize Profit: a * x + b * y;
subject to Constraint1: x + y <= c;

data;
param a := 3;
param b := 2;
param c := 5;

option solver highs;
solve;
display Profit, x, y;

HiGHS 1.10.0: optimal solution; objective 15
0 simplex iterations
0 barrier iterations
Profit = 15
x = 5
y = 0



amplpyを使うと、データをPythonで入力することができる。これによって、Pythonのデータ分析とAMPLの融合が可能になる。

まず、マジックコマンド `%%writefile` で例題を`example.mod` に保存しておく。

In [8]:
%%writefile example.mod
#%%writefile example.mod
param a;
param b;
param c;
var x >= 0;
var y >= 0;
maximize Profit: a * x + b * y;
subject to Constraint1: x + y <= c;

Overwriting example.mod


amplをインストールした環境は`env`に保存されているので、amplのモデルインスタンス`m`を作り、ファイル`example.mod`から読み込んで
モデルインスタンスを生成する。パラメータは`param[パラメータ名]`に代入することによって設定できる。

また、最適化は`solve`メソッドで起動し、最適値やスカラーの変数は`get_value`で得ることができる。

In [5]:
m = AMPL(env)
m.read("example.mod")
m.param["a"] = 3
m.param["b"] = 2
m.param["c"] = 5

m.option["solver"] = "highs"
m.solve()
print("Profit=", m.get_value("Profit"))
print("x=", m.get_value("x"))
print("y=", m.get_value("y"))

HiGHS 1.10.0: optimal solution; objective 15
0 simplex iterations
0 barrier iterations
Profit= 15
x= 5
y= 0


## 多制約ナップサック問題

多制約ナップサック問題を用いて、データをPythonのコマンドで入力する方法を示す。

まず、マジックコマンド `%%writefile` でモデルを`mkp.mod` に保存しておく。


In [9]:
%%writefile  mkp.mod
#%%writefile  mkp.mod
# 多制約ナップサック問題のAMPLモデル
set ITEMS;      # アイテムの集合
set RESOURCES;  # リソースの集合

param value{i in ITEMS} >= 0;    # 各アイテムの価値
param weight{r in RESOURCES, i in ITEMS} >= 0;    # 各アイテムの各リソース消費量
param capacity{r in RESOURCES} >= 0;    # 各リソースの容量

var select{i in ITEMS} binary;    # アイテム選択変数

maximize TotalValue: sum{i in ITEMS} value[i] * select[i];

subject to ResourceCapacity{r in RESOURCES}:
    sum{i in ITEMS} weight[r,i] * select[i] <= capacity[r];


Overwriting mkp.mod


amplインスタンス`m`を生成し、モデルを読み込み、集合、パラメータオプションを設定する。

集合は`set`に**リスト**を代入し、パラメータは`param`に**辞書**を代入し、
ソルバーオプション`option「"solver"]`にソルバー名を代入することによって設定できる。

最適化した後の値（最適値などのスカラー）は`get_value`で、変数の表示は`display`で表示できる。

変数インスタンスは`var`で得ることができ、
`to_dict` (`to_list`, `to_pandas`, `to_string`) で辞書（リスト、データフレーム、文字列）に
変換できる。

In [12]:
import random
random.seed(1)
m = AMPL(env)
m.read("mkp.mod")
m.set["ITEMS"] = [1,2,3]
m.set["RESOURCES"] = ["A","B"]
value = {i:random.randint(1,10) for i in [1,2,3]}
print("value=",value)
m.param["value"] = value
weight = {(r,i):random.randint(5,30) 
                    for r in ["A","B"] for i in [1,2,3]}
print("weight=",weight)
m.param["weight"] = weight
m.param["capacity"] = {r:50 for r in ["A","B"]}

m.option["solver"] = "highs"
m.solve()

print("TotalValue=", m.get_value("TotalValue"))
m.display("select")
m.var["select"].to_dict() #変数を辞書に変換

value= {1: 3, 2: 10, 3: 2}
weight= {('A', 1): 13, ('A', 2): 8, ('A', 3): 20, ('B', 1): 29, ('B', 2): 19, ('B', 3): 20}
HiGHS 1.10.0: optimal solution; objective 13
0 simplex iterations
0 branching nodes
TotalValue= 13
select [*] :=
1  1
2  1
3  0
;



{1: 1, 2: 1, 3: 0}

## 栄養問題

栄養問題を例として、pandasのデータフレームからデータを生成する方法を示す。

In [13]:
%%writefile  diet.mod
#%%writefile  diet.mod
# 栄養問題のAMPLモデル
set FOODS;      # 食品の集合
set NUTRIENTS;  # 栄養素の集合

param cost{f in FOODS} >= 0;    # 各食品の単価
param amount{n in NUTRIENTS, f in FOODS} >= 0;    # 各食品に含まれる栄養素の量
param min_req{n in NUTRIENTS} >= 0;    # 各栄養素の最小必要量
param max_req{n in NUTRIENTS} >= 0;    # 各栄養素の最大許容量

var buy{f in FOODS} >= 0;    # 各食品の購入量

minimize TotalCost: sum{f in FOODS} cost[f] * buy[f];

subject to MinNutrient{n in NUTRIENTS}:
    sum{f in FOODS} amount[n,f] * buy[f] >= min_req[n];

subject to MaxNutrient{n in NUTRIENTS}:
    sum{f in FOODS} amount[n,f] * buy[f] <= max_req[n];

Overwriting diet.mod


データフレームを準備しておく。


In [23]:
import pandas as pd
import numpy as np

food_df = pd.DataFrame(
    [
        ("BEEF", 3.59),
        ("CHK", 2.59),
        ("FISH", 2.29),
        ("HAM", 2.89),
        ("MCH", 1.89),
        ("MTL", 1.99),
        ("SPG", 1.99),
        ("TUR", 2.49),
    ],
    columns=["FOODS", "cost"],
).set_index("FOODS")

# Create a pandas.DataFrame with data for n_min, n_max
nutr_df = pd.DataFrame(
    [
        ("A", 700, 20000),
        ("C", 700, 20000),
        ("B1", 700, 20000),
        ("B2", 700, 20000),
        ("NA", 0, 50000),
        ("CAL", 16000, 24000),
    ],
    columns=["NUTRIENTS", "min_req", "max_req"],
).set_index("NUTRIENTS")

amt_df = pd.DataFrame(
    np.matrix(
        [
            [60, 8, 8, 40, 15, 70, 25, 60],
            [20, 0, 10, 40, 35, 30, 50, 20],
            [10, 20, 15, 35, 15, 15, 25, 15],
            [15, 20, 10, 10, 15, 15, 15, 10],
            [928, 2180, 945, 278, 1182, 896, 1329, 1397],
            [295, 770, 440, 430, 315, 400, 379, 450],
        ]
    ),
    columns=food_df.index.tolist(),
    index=nutr_df.index.tolist(),
)

amt_df

,BEEF,CHK,FISH,HAM,MCH,MTL,SPG,TUR
A,60,8,8,40,15,70,25,60
C,20,0,10,40,35,30,50,20
B1,10,20,15,35,15,15,25,15
B2,15,20,10,10,15,15,15,10
NA,928,2180,945,278,1182,896,1329,1397
CAL,295,770,440,430,315,400,379,450


モデルインスタンス`m`の`set_data`メソッドでデータフレームからデータとインデックス集合を同時に設定できる。
1番目の引数`data`がデータフレームであり、2番目の引数`set_name`がモデルのインデックス集合の名前である。
たとえば、`set_data(data=food_df, set_name= "FOODS")`は、データフレーム`food_df`で定義されるパラメータ`cost`
のデータとそのインデックス集合`FOODS`を設定している。

パラメータ`amount`はデータだけを代入すれば良いので、データフレーム`amt_df`をパラメータインスタンス`m.param["amount"]`に
代入するだけで良い。


In [34]:
m = AMPL(env)
m.read("diet.mod")

m.set_data(data=food_df, set_name= "FOODS")
m.set_data(nutr_df, "NUTRIENTS")
m.param["amount"] = amt_df
m.option["solver"] = "highs"
m.solve()

print("TotalCost=", m.get_value("TotalCost"))
m.display("buy")

HiGHS 1.10.0: optimal solution; objective 90.0041958
2 simplex iterations
0 barrier iterations
TotalCost= 90.00419580419579
buy [*] :=
BEEF   0
 CHK   0
FISH   0
 HAM   0
 MCH  28.6247
 MTL  18.042
 SPG   0
 TUR   0
;



#| hide
## 最適潮流問題

非線形最適化の例として最適潮流問題を解く。ソルバーはipopt（coinモジュール）である。


$$
P_i(V, \delta) = P_i^G - P_i^L, \forall i \in N
$$

$$
Q_i(V, \delta) = Q_i^G - Q_i^L, \forall i \in N
$$

$$
P_i(V, \delta) = V_i \sum_{k=1}^{N}V_k(G_{ik}\cos(\delta_i-\delta_k) + B_{ik}\sin(\delta_i-\delta_k)), \forall i \in N
$$

$$
Q_i(V, \delta) = V_i \sum_{k=1}^{N}V_k(G_{ik}\sin(\delta_i-\delta_k) - B_{ik}\cos(\delta_i-\delta_k)), \forall i \in N
$$

In [ ]:
#| hide
%%writefile pf.mod
# data

set N;                              # set of buses in the network
param nL;                           # number of branches in the network
set L within 1..nL cross N cross N; # set of branches in the network
set GEN within N;                   # set of generator buses
set REF within N;                   # set of reference (slack) buses
set PQ within N;                    # set of load buses
set PV within N;                    # set of voltage-controlled buses
set YN :=                           # index of the bus admittance matrix
    setof {i in N} (i,i) union
    setof {(i,k,l) in L} (k,l) union
    setof {(i,k,l) in L} (l,k);

# bus data

param V0     {N}; # initial voltage magnitude
param delta0 {N}; # initial voltage angle
param PL     {N}; # real power load
param QL     {N}; # reactive power load
param g_s    {N}; # shunt conductance
param b_s    {N}; # shunt susceptance


# generator data

param PG {GEN}; # real power generation
param QG {GEN}; # reactive power generation

# branch indexed data

param T    {L}; # initial voltage ratio
param phi  {L}; # initial phase angle
param R    {L}; # branch resistance
param X    {L}; # branch reactance
param g_sh {L}; # shunt conductance
param b_sh {L}; # shunt susceptance

param g {(l,k,m) in L} := R[l,k,m]/(R[l,k,m]^2 + X[l,k,m]^2);  # series conductance
param b {(l,k,m) in L} := -X[l,k,m]/(R[l,k,m]^2 + X[l,k,m]^2); # series susceptance

# bus admittance matrix real part
param G {(i,k) in YN} =
    if (i == k) then (
        g_s[i] +
        sum{(l,i,u) in L} (g[l,i,u] + g_sh[l,i,u]/2)/T[l,i,u]**2 +
        sum{(l,u,i) in L} (g[l,u,i] + g_sh[l,u,i]/2)
    )
    else (
        -sum{(l,i,k) in L} ((
            g[l,i,k]*cos(phi[l,i,k])-b[l,i,k]*sin(phi[l,i,k])
        )/T[l,i,k]) -
        sum{(l,k,i) in L} ((
            g[l,k,i]*cos(phi[l,k,i])+b[l,k,i]*sin(phi[l,k,i])
        )/T[l,k,i])
    );

# bus admittance matrix imaginary part
param B {(i,k) in YN} =
    if (i == k) then (
        b_s[i] +
        sum{(l,i,u) in L} (b[l,i,u] + b_sh[l,i,u]/2)/T[l,i,u]**2 +
        sum{(l,u,i) in L} (b[l,u,i] + b_sh[l,u,i]/2)
    )
    else (
        -sum{(l,i,k) in L} (
            g[l,i,k]*sin(phi[l,i,k])+b[l,i,k]*cos(phi[l,i,k])
        )/T[l,i,k] -
        sum{(l,k,i) in L} (
            -g[l,k,i]*sin(phi[l,k,i])+b[l,k,i]*cos(phi[l,k,i])
        )/T[l,k,i]
    );

# variables
var V     {i in N} := V0[i];     # voltage magnitude
var delta {i in N} := delta0[i]; # voltage angle

# real power injection
var P {i in N} =
    V[i] * sum{(i,k) in YN} V[k] * (
        G[i,k] * cos(delta[i] - delta[k]) +
        B[i,k] * sin(delta[i] - delta[k])
    );

# reactive power injection
var Q {i in N} =
    V[i] * sum{(i,k) in YN} V[k] * (
        G[i,k] * sin(delta[i] - delta[k]) -
        B[i,k] * cos(delta[i] - delta[k])
    );

# constraints

s.t. p_flow {i in (PQ union PV)}:
    P[i] == (if (i in GEN) then PG[i] else 0) - PL[i];

s.t. q_flow {i in PQ}:
    Q[i] == (if (i in GEN) then QG[i] else 0) - QL[i];

s.t. fixed_angles {i in REF}:
    delta[i] == delta0[i];

s.t. fixed_voltages {i in (REF union PV)}:
    V[i] == V0[i];

Writing pf.mod


In [ ]:
#| hide
df_bus = pd.DataFrame(
    [
        [1, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0],
        [2, 0.0, 0.0, 0.0, 0.3, 0.95, 1.05],
        [3, 0.0, 0.0, 0.05, 0.0, 0.95, 1.05],
        [4, 0.9, 0.4, 0.0, 0.0, 0.95, 1.05],
        [5, 0.239, 0.129, 0.0, 0.0, 0.95, 1.05],
    ],
    columns=["bus", "PL", "QL", "g_s", "b_s", "V_min", "V_max"],
).set_index("bus")

df_branch = pd.DataFrame(
    [
        [1, 1, 2, 0.0, 0.3, 0.0, 0.0, 1.0, 0.0],
        [2, 1, 3, 0.023, 0.145, 0.0, 0.04, 1.0, 0.0],
        [3, 2, 4, 0.006, 0.032, 0.0, 0.01, 1.0, 0.0],
        [4, 3, 4, 0.02, 0.26, 0.0, 0.0, 1.0, -3.0],
        [5, 3, 5, 0.0, 0.32, 0.0, 0.0, 0.98, 0.0],
        [6, 4, 5, 0.0, 0.5, 0.0, 0.0, 1.0, 0.0],
    ],
    columns=["row", "from", "to", "R", "X", "g_sh", "b_sh", "T", "phi"],
).set_index(["row", "from", "to"])

gen = [1, 3, 4]
ref = [1]
pq = [3, 4]
pv = [2, 5]

# print(df_bus)
# print(df_branch)
# data preprocessing

ampl_bus = pd.DataFrame()
cols = ["PL", "QL", "g_s", "b_s"]
for col in cols:
    ampl_bus.loc[:, col] = df_bus.loc[:, col]
ampl_bus["V0"] = 1.0
ampl_bus["delta0"] = 0.0

ampl_branch = pd.DataFrame()
ampl_branch = df_branch.copy()

ampl_gen = pd.DataFrame()
ampl_gen.index = gen
ampl_gen["PG"] = 0.0
ampl_gen["QG"] = 0.0

from math import radians, degrees

# convert degrees to radians
ampl_bus["delta0"] = ampl_bus["delta0"].apply(radians)
ampl_branch["phi"] = ampl_branch["phi"].apply(radians)

print(ampl_bus)
print(ampl_branch)
print(ampl_gen)


        PL     QL   g_s  b_s   V0  delta0
bus                                      
1    0.000  0.000  0.00  0.0  1.0     0.0
2    0.000  0.000  0.00  0.3  1.0     0.0
3    0.000  0.000  0.05  0.0  1.0     0.0
4    0.900  0.400  0.00  0.0  1.0     0.0
5    0.239  0.129  0.00  0.0  1.0     0.0
                 R      X  g_sh  b_sh     T      phi
row from to                                         
1   1    2   0.000  0.300   0.0  0.00  1.00  0.00000
2   1    3   0.023  0.145   0.0  0.04  1.00  0.00000
3   2    4   0.006  0.032   0.0  0.01  1.00  0.00000
4   3    4   0.020  0.260   0.0  0.00  1.00 -0.05236
5   3    5   0.000  0.320   0.0  0.00  0.98  0.00000
6   4    5   0.000  0.500   0.0  0.00  1.00  0.00000
    PG   QG
1  0.0  0.0
3  0.0  0.0
4  0.0  0.0


In [ ]:
#| hide
def pf_run(bus, branch, gen, ref, pq, pv):
    # initialyze AMPL and read the model
    ampl = AMPL(env)
    ampl.read("pf.mod")

    # load the data
    ampl.set_data(bus, "N")
    ampl.param["nL"] = branch.shape[0]
    ampl.set_data(branch, "L")
    ampl.set_data(gen, "GEN")
    ampl.set["REF"] = ref
    ampl.set["PQ"] = pq
    ampl.set["PV"] = pv

    ampl.solve(solver="ipopt")

    solve_result = ampl.get_value("solve_result")
    if solve_result != "solved":
        print("WARNING: solver exited with %s status." % (solve_result,))

    return ampl.get_data("V", "delta").to_pandas(), solve_result


df_res, solver_status = pf_run(ampl_bus, ampl_branch, ampl_gen, ref, pq, pv)

# convert radians back to degrees
df_res["delta"] = df_res["delta"].apply(degrees)

# print results
print("solver status:", solver_status)
print(df_res)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       30
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       18

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot